Feb 1, 2019 - This script imports a L2 geotiff files from USGS. Units of the USGS product is Reflectance (R). Files include all cloud free images through the year from 1982 to 2018 (n=88) to run through MESMA (Multiple Endmember Spectral Mixture Analysis - developed by Roberts et al) based on basic code shared by Tom Bell.

The script uses gdal.Open().ReadAsArray() to load subset ENVI files (created from Dennis Finger handmade ROI file) as an array into python.

MESMA algorithim is run on each image to determine the fraction of kelp per pixel. The percent kelp threshold for a single pixel is 20% (0.2). If a pixel contains >=0.2 and <=1, the fractional area is calculated by taking the product of the MESMA kelp fraction and the pixel area (900m2). Currently, this threshold value is not validated. 

Oct. 9, 2019 - The script was modified to get Latitude and Longitude information from the envi files and converted into a numpy array. 
Nov. 11, 2019 - The script was modified to use new ROI for Mendocino and Sonoma Counties and using external HD rather than Google Drive. 


In [1]:
import os
import sys
import time
import glob
import gdal
import rasterio
import pandas as pd
from affine import Affine
from pyproj import Proj, transform
from rasterio.plot import show
import numpy as np
from scipy.stats.mstats import gmean
from scipy.stats import linregress
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from datetime import datetime
import concurrent.futures
from multiprocessing import cpu_count

In [6]:
# change dir to Bull Kelp drive
#os.chdir('')

filelist = glob.glob('Landsat_imagery/merged/*_subset.tif')
filelist.sort()


In [ ]:
# Get Lat/Lon array from one of the envi files - they should all be the same
# source: 
# https://gis.stackexchange.com/questions/129847/obtain-coordinates-and-corresponding-pixel-values-from-geotiff-using-python-gdal


# Read raster
with rasterio.open(filelist[0]) as r:
    T0 = r.transform  # upper-left pixel corner affine transform
    p1 = Proj(r.crs)
    A = r.read()  # pixel values

# All rows and columns
cols, rows = np.meshgrid(np.arange(A.shape[2]), np.arange(A.shape[1]))

# Get affine transform for pixel centres
T1 = T0 * Affine.translation(0.5, 0.5)
# Function to convert pixel row/column index (from 0) to easting/northing at centre
rc2en = lambda r, c: (c, r) * T1

# All eastings and northings (there is probably a faster way to do this)
eastings, northings = np.vectorize(rc2en, otypes=[np.float, np.float])(rows, cols)

# Project all longitudes, latitudes
p2 = Proj(proj='latlong',datum='WGS84')
longs, lats = transform(p1, p2, eastings, northings)

# find min and max UTM coordinates of ROI region
minX = np.min(eastings)
maxX = np.max(eastings)
minY = np.min(northings)
maxY = np.max(northings)

minLon = np.min(longs)
maxLon = np.max(longs)
minLat = np.min(lats)
maxLat = np.max(lats)

np.savetxt('Image_lats.txt',lats)
np.savetxt('Image_longs.txt',longs)

In [ ]:
# create empty array of zeros for MESMA summary of MESMA output data
now = datetime.now()
sp = 'nereo'

# create an array to fill from mesma (year, month, day)
kelp_output = pd.DataFrame(columns = ['Year','Imagedate1','Imagedate2','Area(km2)','FractionalSum'])

# Define the fractional range of the model
kelp_lb = 0.2
kelp_ub = 1.0

frac_range = 'MESMA fraction range = %0.1f - %0.1f' %(kelp_lb,kelp_ub)
print(frac_range)

# load txt file with standard locations for water
water_file = np.loadtxt('waterEMv2.csv', 
                        delimiter = ',',
                        skiprows = 9)

t1 = time.perf_counter()

# ---- run as loop or with concurrent futures module

#for filename in filelist:
def mesma(filename):
    #t2 = time.perf_counter()
    print('running...',filename)
    
    # choose a name to save the file as later in the script
    saveas = filename[71:98]
    image1 = saveas[10:18]
    image2 = saveas[19:28]
    date_string  = f'{datetime.now():%Y%m%d}'
    
    # open the 4 band raster using GDAL
    # problems getting Jupyterlabs to open the raster using gdal, so gdal.USeExceptions to get around
    # the problem of NoneType error
    gdal.UseExceptions()
    ds=None
    open_array = gdal.Open(filename).ReadAsArray()[0:4,:,:]
    #print('image opened')

    
    #ref_offset = np.min(open_array)
    #ref_offset = open_array[open_array > 0].min()
    
    #Get dimensions of the array
    nrows = open_array.shape[1]
    ncols = open_array.shape[2]
    nbands = len(open_array)

    ###############################################################################
    ###### THIS IS A TRANSLATION OF TOM BELL'S MATLAB MESMA CODE INTO PYTHON ######
    ###############################################################################

    # This is the kelp endmember Tom Bell has used for all his work; this refelectance
    # is syptical of brown algae and shouldn't have to be change for nereo; this 
    # represents refelctance in digital numbers for the blue, green, red, and NIR 
    # bands for Landsat

    # bull kelp endmember created using EAR in ENVI ViperTool
    kelp = np.array([[344,383,275,920]]).reshape((4,1))# - ref_offset
    
    # Create a bunch of constants and empty arrays to populate with MESMA results
    water_comp = np.zeros((len(water_file),nbands))
    frac_water = np.zeros((len(water_file),nrows,ncols))
    frac_kelp = np.zeros((len(water_file),nrows,ncols))
    frac_total = np.zeros((len(water_file),nrows,ncols))
    rmse = np.zeros((len(water_file),nrows,ncols))
    best_rmse = np.zeros((nrows,ncols))
    location = np.zeros((nrows,ncols))
    best_frac_kelp = np.zeros((nrows,ncols))
    pixel_frac_area = np.zeros((nrows,ncols))
    best_frac_total = np.zeros((nrows,ncols))
    
    
    for n in range(len(water_file)):
        
        resids = np.zeros((nbands,nrows,ncols))
        
        water_comp[n,:] = open_array[:,int(water_file[n,1]),int(water_file[n,0])] #- ref_offset
        water = water_comp[n,:].reshape((4,1))
        
        if np.any( water < 0 ):
            water_comp[n,:] = np.NaN
            frac_total[n,:,:] = np.NaN
            frac_kelp[n,:,:] = np.NaN
            rmse[n,:,:] = np.NaN  
            #continue
            
        else:
            # Here, can create an array of one of your seawater endmembers and kelp 
            # endmembers.  The seawater endmember will be one of 30 (or so) endmembers 
            # you choose from deep water ocean pixels in each image
            E = np.matrix(np.hstack((water, kelp)))

            # Here, perform a singular value decomposition of the endmember array. 
            # Singular  value decomposition is 
            # In the matlab code, Tom used 0 to produce an economy size  decomposition. 
            # I've put full_matrices=False, which I believe is the correct syntax in Python
            U,S,V = np.linalg.svd(E, full_matrices=False) 

            # Here, divide the V (eigenvector) term by the S term (basically eigenvalues)
            # and multiply the IS term by the transposed U matrix
            em_inv = V/S * U.T #(2,4)   

            for r in range(nrows):
                for c in range(ncols):
                    # Now multiply this em_inv matrix by pixel reflectance to obtain 
                    # the fraction of seawater and fraction of kelp in the pixel
                    image_band = open_array[:,r,c].reshape((4,1))

                    if image_band[0] > 0.0:
                        #print(image_band)
                        frac_calc = em_inv * image_band
                        frac_water[n,r,c] = frac_calc[0,0]
                        frac_kelp[n,r,c] = frac_calc[1,0]

                        # filter out kelp fractons greater than 1 and less than 0.2, and less than water fraction
                        if np.logical_and(np.logical_and(frac_kelp[n,r,c]<=1.0,frac_kelp[n,r,c]>=0.2),frac_kelp[n,r,c]>frac_water[n,r,c]):
                            frac_total[n,r,c] = frac_water[n,r,c]+frac_kelp[n,r,c]
                            #print('frac water+kelp = ',frac_total[n,r,c])

                            # Obtain modeled reflectance by multiplying endmembers by the fractions.
                            model_calc = E[:,0]*frac_water[n,r,c] + E[:,1]*frac_kelp[n,r,c]

                            # subtract modeled reflectance from pixel reflectance to 
                            # obtain residuals across the 4 bands. Divide by 10000 to put the 
                            # RMSE into more relatable terms
                            for z in range(nbands):
                                resids[z,r,c] = (image_band[z] - model_calc[z])/10000

                            # Once you choose the best model you ouput the fractions associated with it.

                            # This is calculating for every pixel within the ROI
                            rmse[n,r,c] = np.sqrt(sum(resids[:,r,c]**2)/4)
                            if rmse[n,r,c]>0.01:
                                rmse[n,r,c] = np.NaN
                        else:
                            frac_total[n,r,c] = np.NaN
                            frac_kelp[n,r,c] = np.NaN
                            rmse[n,r,c] = np.NaN

    print('mesma complete')
    
    for r in range(nrows):
        for c in range(ncols):
            try:
                get_loc = np.unravel_index(np.nanargmin(rmse[:,r,c]), rmse.shape)
                location[r,c] = get_loc[2]
                
                # final output is using the frac_kelp from the best rmse score for each pixel
                best_rmse[r,c] = rmse[int(location[r,c]),r,c]
                best_frac_kelp[r,c] = frac_kelp[int(location[r,c]),r,c]
                best_frac_total[r,c] = frac_total[int(location[r,c]),r,c]
                pixel_frac_area[r,c] = best_frac_kelp[r,c]*900
                
            except ValueError:
                get_loc = (0,0,0)
                location[r,c] = get_loc[2]

    frac_area = np.nansum(pixel_frac_area)
    frac_sum = np.nansum(best_frac_kelp)
    #print('sum of fractional area (km2) =', frac_area/1000000)
    #print('sum of kelp fractions = ', frac_sum)
    
    # get location and kelp fraction output
    kelp_loc = np.where(np.nan_to_num(best_frac_kelp, nan=0.0)>0.0)
    
    kelp_northings = np.array(northings[kelp_loc[0],kelp_loc[1]]).reshape(-1,1)
    kelp_lats = np.array(lats[kelp_loc[0],kelp_loc[1]]).reshape(-1,1)
    kelp_eastings = np.array(eastings[kelp_loc[0],kelp_loc[1]]).reshape(-1,1)
    kelp_lons = np.array(longs[kelp_loc[0],kelp_loc[1]]).reshape(-1,1)
    kelp_vals = np.array(best_frac_kelp[kelp_loc[0],kelp_loc[1]]).reshape(-1,1)
    kelp_annual_loc = np.hstack((kelp_northings,kelp_lats,kelp_eastings,kelp_lons,kelp_vals))
    
    # -- Plot summary plots for each image
    
    fig, (ax1, ax2, ax3) = plt.subplots(ncols=3,figsize=(11,4))
    cmap = plt.cm.jet
    cmap.set_bad(color='gainsboro')
    im = ax1.imshow(best_frac_kelp, aspect='auto',cmap=cmap)
    ax1.text(0.90, 0.8, 'kelp frac area \n = \n {0:.3f} km2'.format(frac_area/1000000),
        verticalalignment='bottom', horizontalalignment='right',
        transform=ax1.transAxes,
        color='white', fontsize=11)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax)
    ax2.plot(water_comp.T)
    ax2.set(xlabel = 'Band number',
            ylabel = 'Water Endmember Reflectance',
            xticks = np.arange(1,5,1))
    ax3.hist(best_frac_kelp[~np.isnan(best_frac_kelp)],bins=[0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
    ax3.set(xlabel = 'MESMA kelp fraction',
            ylabel = 'frequency')
    fig.suptitle('LS_{0}_PyMESMA_{1}'.format(sp,saveas))
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

    
    
    # ----- add data to the summary output array and save as textfile
    
    kelp_area = f'{saveas} fractional area = {frac_area/1000000} km2' # convert m2 to km2
    kelp_area = f'{saveas} fractional sum = {frac_sum}'
    print(kelp_area)
    
    print(f'{saveas} saving files...')
    
    np.savetxt(f'LS_PyMESMA_{saveas}_{date_string}.csv',
                    kelp_annual_loc, 
                    delimiter=",", 
                    fmt = '%i %0.7f %i %0.7f %0.5f',
                    header = f'Python MESMA for {saveas} run on {date_string} \n {frac_range} \n Northings (y); Latitude; Eastings (x) Longitude; Kelp Fraction')

    #for later processing of autocorrelation, etc.
    np.savetxt(f'LS_PyMESMAfrac_max_{saveas}_{date_string}.txt',
               best_frac_kelp,
               fmt = '%0.3f')
    
    np.savetxt(f'LS_PyMESMAarea_max_{saveas}_{date_string}.txt',
               pixel_frac_area,
               fmt = '%0.3f')

    # annual summary file
    kelp_output = kelp_output.append({'Year': int(filename[90:94]),
                       'Imagedate1': int(image1),
                       'Imagedate2': int(image2),
                       'Area(km2)': frac_area/1000000,
                       'FractionalSum': frac_sum},ignore_index=True)

    

    # ----- output the data into .txt files
    now = datetime.now() 
    save_now = now.strftime('%Y%m%d_%H%M')
    np.savetxt(f'LS_PyMESMA_McPhersonROI_area_{save_now}.txt',
               kelp_output,
               fmt = '%i %i %i %0.5f %0.5f',
               header = f'Python MESMA results of Mendocino and Sonoma Counties for all years run on {save_now} \n {frac_range} \n Year Filename1 Filename2 Area(km^2) Fractional_Sum')


with concurrent.futures.ProcessPoolExecutor(max_workers=12) as executor:
    executor.map(mesma, filelist)
        
    
t4 = time.perf_counter()

print(f'finished in {t4-t1} seconds')


